In [1]:
# !pip3 install flickrapi

In [ ]:
# from flickrapi import FlickrAPI
# import os
# import urllib.request

# # Working directory
# os.chdir('C:/Users/rxcov/Dropbox/M.Sc. Studium/2. Semester/ADS - Applied Data Science/food-classifier/food/')
# path = 'C:/Users/rxcov/Dropbox/M.Sc. Studium/2. Semester/ADS - Applied Data Science/food-classifier/food/'
# print(os.getcwd())

# # Keys for flickr API
# FLICKR_PUBLIC = 'c6a2c45591d4973ff525042472446ca2'
# FLICKR_SECRET = '202ffe6f387ce29b'

# # Settings (extras = different resolution of photos)
# flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
# extras = 'url_s'

# foods = ['grapes',
#  'salad',
#  'lasagna',
#  'pancakes',
#  'sushi',
#  'ramen',
#  'tacos',
#  'pizza',
#  'steak',
#  'risotto']

        
# for food in foods:
    
#     try:
#         os.mkdir(path + str(food))
#     except OSError as e:
#         if e.errno != errno.EEXIST:
#             raise
        
#     # Get information about photos using the flickr API
#     cont = flickr.photos.search(text=food, per_page=100, extras=extras)
#     photos = cont['photos']
#     # Object type
#     print(type(photos))
    
#     title = 0

#     for i in photos["photo"]:
#         url = i["url_s"]
#         title = title+1
#         urllib.request.urlretrieve(url,str(food) +"/"+ str(food) + str(title)+'.jpg')

## Google Images Scraper

Saves n number of images for specific search query and saves them in same repository (in this case under images)
Source: https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d
Flickrapi will not be used any further, as quality of pictures is to bad for feeding the classification model.

In [12]:
!pip3 install selenium

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [22]:
!pip3 install Pillow
import io
from PIL import Image
import hashlib


def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
      
            
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [14]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [25]:
foods = ['grapes',
 'salad',
 'lasagna',
 'pancakes',
 'sushi',
 'ramen',
 'tacos',
 'pizza',
 'steak',
 'risotto']

for food in foods:
    search_and_download(
        search_term=food,
        driver_path='chromedriver.exe',
        target_path='./food'
        number_images=2)

Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/6/6c/Abhar-iran.JPG - as ./images\grapes\1d3a05494b.jpg
SUCCESS - saved https://www.morethanshipping.com/wp-content/uploads/2020/08/us-leader-in-grapes-mts-2020.jpg - as ./images\grapes\4afcf1fac8.jpg
Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
SUCCESS - saved https://www.sonachgefuehl.de/wp-content/uploads/2019/05/Caesar_Salad_01.jpg - as ./images\salad\aa9878a252.jpg
SUCCESS - saved https://image.essen-und-trinken.de/11952552/t/QD/v10/w960/r1/-/eut-mit-thermomix-201802016-chicken-caesar-salad-jpg--65187-.jpg - as ./images\salad\9da30d839b.jpg
Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
SUCCESS - saved https://img.chefkoch-cdn.de/rezepte/1576041265136617/bilder/823227/crop-960x720/risotto-grundrezept-mit-vier-abwandlungen.jpg - as ./images\risotto\5da7adc986.jpg
SU